#pip optuna

In [ ]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.2 MB/s eta 0:00:00


#Libraries

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, roc_curve, precision_recall_curve, auc
)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout , Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import backend as K

import optuna



#Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load the data
import joblib
data = joblib.load('/content/drive/MyDrive/Bankruptcy Prediction Data/Prepared Dataset for Thesis/Dataset.pkl')

In [ ]:
X = data.drop(columns=['result'])
y = data['result']

In [ ]:
data.shape

(255919, 61)

In [ ]:

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


#Models

#training

defining f1 score

In [ ]:
import tensorflow as tf

def f1_score_metric(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.round(y_pred)
    y_pred = tf.cast(y_pred, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    fn = tf.reduce_sum(y_true * (1 - y_pred))
    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())
    f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1


#Optuna training section

In [ ]:
def create_model(trial):
    model = Sequential()

    #input
    model.add(Input(shape=(X_train.shape[1],)))  # Define the input shape
    model.add(Dense(trial.suggest_int('units_1', 16, 128, step=16), activation='relu'))
    model.add(Dropout(trial.suggest_float('dropout_1', 0.2, 0.5, step=0.1)))

    n_layers = trial.suggest_int('n_layers', 1, 3)  # modify layers here
    for i in range(1, n_layers + 1):
        model.add(Dense(trial.suggest_int(f'units_{i+1}', 16, 128, step=16), activation='relu'))
        model.add(Dropout(trial.suggest_float(f'dropout_{i+1}', 0.2, 0.5, step=0.1)))
    #output
    model.add(Dense(1, activation='sigmoid'))

    #learningrate
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[f1_score_metric])
    return model

def objective(trial):
    model = create_model(trial)

    # model
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=20,
        batch_size=256,
        verbose=1
    )

    # Predict probabilities
    val_proba = model.predict(X_train[int(0.8 * len(X_train)):])
    val_true = y_train[int(0.8 * len(y_train)):]

    # F1 best threshold
    best_threshold = 0.5
    best_f1 = 0
    for threshold in np.arange(0.1, 1.0, 0.01):
        val_pred = (val_proba.flatten() >= threshold).astype(int)
        f1 = f1_score(val_true, val_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    trial.set_user_attr("best_threshold", best_threshold)
    return best_f1

# Create an Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=3600)

print("Best hyperparameters:")
print(study.best_params)

best_trial = study.best_trial
best_threshold = best_trial.user_attrs["best_threshold"]
print(f"Best Threshold: {best_threshold}")

best_params = study.best_params
final_model = create_model(study.best_trial)
final_model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=32, verbose=1)
y_test_proba = final_model.predict(X_test).flatten()
y_test_pred = (y_test_proba >= best_threshold).astype(int)
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")


[I 2024-12-27 17:06:46,741] A new study created in memory with name: no-name-4ae4318d-f736-4b91-88c0-cef200c07978


800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step


[I 2024-12-27 17:07:18,282] Trial 0 finished with value: 0.7250776129699896 and parameters: {'units_1': 128, 'dropout_1': 0.30000000000000004, 'n_layers': 2, 'units_2': 48, 'dropout_2': 0.4, 'units_3': 112, 'dropout_3': 0.2, 'learning_rate': 0.0005452428859468979}. Best is trial 0 with value: 0.7250776129699896.


800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


[I 2024-12-27 17:07:46,647] Trial 1 finished with value: 0.705530116238112 and parameters: {'units_1': 112, 'dropout_1': 0.4, 'n_layers': 2, 'units_2': 32, 'dropout_2': 0.4, 'units_3': 96, 'dropout_3': 0.2, 'learning_rate': 0.00014628453509763737}. Best is trial 0 with value: 0.7250776129699896.


...

800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


[I 2024-12-27 17:27:54,859] Trial 48 finished with value: 0.7177081588205729 and parameters: {'units_1': 48, 'dropout_1': 0.4, 'n_layers': 1, 'units_2': 80, 'dropout_2': 0.30000000000000004, 'learning_rate': 0.0008864545002372614}. Best is trial 11 with value: 0.7275641025641025.


800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


[I 2024-12-27 17:28:18,733] Trial 49 finished with value: 0.7199469408058365 and parameters: {'units_1': 96, 'dropout_1': 0.4, 'n_layers': 2, 'units_2': 112, 'dropout_2': 0.30000000000000004, 'units_3': 96, 'dropout_3': 0.4, 'learning_rate': 0.002992971946231379}. Best is trial 11 with value: 0.7275641025641025.


Best hyperparameters:
{'units_1': 128, 'dropout_1': 0.2, 'n_layers': 1, 'units_2': 96, 'dropout_2': 0.5, 'learning_rate': 0.0006448191978189127}
Best Threshold: 0.3899999999999999
Epoch 1/20
3199/3199 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - f1_score_metric: 0.4822 - loss: 0.2282 - val_f1_score_metric: 0.5244 - val_loss: 0.1749
Epoch 2/20
3199/3199 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - f1_score_metric: 0.5820 - loss: 0.1772 - val_f1_score_metric: 0.5990 - val_loss: 0.1604
Epoch 3/20
3199/3199 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - f1_score_metric: 0.6099 - loss: 0.1646 - val_f1_score_metric: 0.5781 - val_loss: 0.1606
Epoch 4/20
3199/3199 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - f1_score_metric: 0.6035 - loss: 0.1600 - val_f1_score_metric: 0.5954 - val_loss: 0.1583
Epoch 5/20
3199/3199 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - f1_score_metric: 0.6182 - loss: 0.1578 - val_f1_score_metric: 0.6131 - val_loss: 0.1563
Epoch 6/20
3199/3199 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - f1_score_metric: 0.6267 - loss: 0.1573 - va